In [1]:
'''
pip install --upgrade jupyter_client
'''
import torch
import torchreid
torchreid.models.show_avai_models()

/home/user01/anaconda3/envs/reid/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [2]:
from torchreid import models, utils

In [3]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=32
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset


/home/user01/data/track/deep-person-reid/torchreid/data/datasets/image/market1501.py:38: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  'The current data structure is deprecated. Please '


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [4]:
model = models.build_model(name='osnet_x1_0',
                           num_classes=datamanager.num_train_pids,
                           loss='softmax')

model = model.cuda()

Successfully loaded imagenet pretrained weights from "/home/user01/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [5]:
# New layer "classifier" has a learning rate of 0.01
# The base layers have a learning rate of 0.001
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='sgd',
    lr=0.01,
    # staged_lr=True,
    # new_layers='classifier',
    # base_lr_mult=0.1
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

In [6]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler
)

engine.run(
    save_dir='log/osnet_x1_0',
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=False,
    fixbase_epoch=5,
    open_layers='classifier'
)
# or open_layers=['fc', 'classifier'] if there is another fc layer that
# is randomly initialized, like resnet50_fc512

=> Start training
* Only train classifier (epoch: 1/5)
epoch: [1/60][10/404]	time 0.019 (0.193)	data 0.000 (0.026)	eta 1:18:07	loss 6.5961 (6.6804)	acc 0.0000 (0.0000)	lr 0.010000
epoch: [1/60][20/404]	time 0.016 (0.105)	data 0.000 (0.013)	eta 0:42:28	loss 6.6491 (6.6611)	acc 3.1250 (0.7812)	lr 0.010000
epoch: [1/60][30/404]	time 0.018 (0.077)	data 0.000 (0.009)	eta 0:30:55	loss 6.4064 (6.6767)	acc 0.0000 (0.6250)	lr 0.010000
epoch: [1/60][40/404]	time 0.017 (0.062)	data 0.000 (0.007)	eta 0:24:55	loss 6.8855 (6.6581)	acc 3.1250 (1.1719)	lr 0.010000
epoch: [1/60][50/404]	time 0.031 (0.054)	data 0.001 (0.005)	eta 0:21:37	loss 6.5928 (6.6406)	acc 0.0000 (1.5000)	lr 0.010000
epoch: [1/60][60/404]	time 0.022 (0.048)	data 0.000 (0.005)	eta 0:19:11	loss 6.3882 (6.6225)	acc 3.1250 (1.6667)	lr 0.010000
epoch: [1/60][70/404]	time 0.017 (0.044)	data 0.000 (0.004)	eta 0:17:43	loss 6.2499 (6.5851)	acc 3.1250 (1.7411)	lr 0.010000
epoch: [1/60][80/404]	time 0.016 (0.041)	data 0.000 (0.003)	eta 0:16:2